**Learning outcomes**

* Define the Hadamard transform.
* Understand the effect of applying an oracle between Hadamard transforms.

In [3]:
import numpy as np
import pennylane as qml

In [4]:
def oracle_matrix(combo):
    """Return the oracle matrix for a secret combination.

    Args:
        combo (list[int]): A list of bits representing a secret combination.

    Returns:
        array[float]: The matrix representation of the oracle.
    """
    index = np.ravel_multi_index(combo, [2]*len(combo)) # Index of solution
    my_array = np.identity(2**len(combo)) # Create the identity matrix
    my_array[index, index] = -1

    return my_array

**Codercise A.5.1.** Implement the circuit above, consisting of a Hadamard transform, an oracle, and a Hadamard transform. The oracle is provided as `oracle_matrix(combo)`, which you can invoke using the `QubitUnitary()` function.

Tip. To implement the Hadamard transform, apply PennyLane's `broadcast()` function instead of a `for` loop. This applies a unitary multiple times according to a given pattern, and for a specified set of wires.

In [5]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def hoh_circuit(combo):
    """A circuit which applies Hadamard-oracle-Hadamard and returns probabilities.

    Args:
        combo (list[int]): A list of bits representing a secret combination.

    Returns:
        list[float]: Measurement outcome probabilities.
    """
    oracle = oracle_matrix(combo)

    qml.broadcast(qml.Hadamard, wires=[i for i in range(n_bits)], pattern="single")
    qml.QubitUnitary(oracle, wires=[i for i in range(n_bits)])
    qml.broadcast(qml.Hadamard,pattern="single",wires=[i for i in range(n_bits)])

    return qml.probs(wires=range(n_bits))

![circuit](./images/A.5.1.png)

We finally have something noticeably different from the uniform distribution. But curiously, whatever the secret combination, the probability distribution is the same, with a big peak at $0$ and a small probability of observing any other state. Since the output of the circuit doesn't change, we cannot use it to learn the secret combination! But perhaps, like the pair-testing algorithm applied to a lock with multiple solutions, we can learn something else.